In [1]:
# schedule N courses in K time slots in H exam halls
# Fasih Abdullah i200432 CS-F Assignment 2
from copy import deepcopy                        
import random                                    
from math import exp                             
import numpy as np


In [2]:
N = 5 # number of courses
K = 3 # number of time slots
H = 2  # number of exam halls
courseDuration = [random.randint(2,4) for _ in range(N)]  # duration of courses
hallHours = [random.randint(7,10) for _ in range(H)]   # hours per day for exam halls
conflicts_With_StudentCount = [(1,2,10),(1,4,5),(2,5,7),(3,4,12),(4,5,8)]   #nummber of conflicts between courses (i,j,conflicts)
conflicts = [(i,j) for (i,j,_) in conflicts_With_StudentCount]  #number of conflicts between courses (i,j)
generationLimit = 100 #maximum number of generations
minimumSolution = []

In [3]:
print("hallHours: ", hallHours)
print("conflicts: ", conflicts)
print("conflicts_With_StudentCount: ", conflicts_With_StudentCount)

hallHours:  [10, 10]
conflicts:  [(1, 2), (1, 4), (2, 5), (3, 4), (4, 5)]
conflicts_With_StudentCount:  [(1, 2, 10), (1, 4, 5), (2, 5, 7), (3, 4, 12), (4, 5, 8)]


In [4]:
# generate a random schedule
def generateSchedule(): # [(course, time slot, hall)]
    return [(i+1, random.randint(1,K), random.randint(1,H)) for i in range(N)]

# generate a random population of size
def generatePopulation(size):
    return [generateSchedule() for _ in range(size)]

# print population line by line
def printPopulation(population):
    for schedule in population:
        print(schedule)
        
# check  reoccurance of a schedule in a population
def reoccurance(schedule):
    for i in range(len(schedule)):
        (course, time, hall) = schedule[i]
        for j in range(len(schedule)):
            if i == j:
                continue
            else:
                (course2, time2, hall2) = schedule[j]
                if course == course2 and time == time2 and hall == hall2:
                    return True                
    return False

# check course overlap in a schedule i.e same course in same time slot and hall penalty = 500
def overlap(schedule):
    overlapValue = 0
    for i in range(len(schedule)):
        (course, time, hall) = schedule[i]
        for j in range(i+1,len(schedule)):
            (course2, time2, hall2) = schedule[j]
            # if course!=course2 and time == time2 and hall == hall2:
            #     overlapValue += 500    
            if course!=course2 and hall == hall2 and time2 in range(time,time+courseDuration[course-1]):
                overlapValue += 500            
    return overlapValue

#  calculate the fitness of a schedule based on conflicts and hallHours
def fitness(schedule,show=False):
    fitnessValue = 0
    for (i,j) in conflicts: # checks conflicts between courses
        for (course, time, hall) in schedule:
            if course == i:
                for (course2, time2, hall2) in schedule:
                    if course2 == j:
                        if time == time2 and hall == hall2:
                            fitnessValue += 100
                            if show:
                                print("Conflict: ", i, j, "time: ", time, "hall: ", hall, "New Fitness: ", fitnessValue)
    if reoccurance(schedule): # checks reoccurances of courses in a schedule
        fitnessValue += 1000
    if overlap(schedule): # checks overlap of courses in a schedule
        fitnessValue += overlap(schedule)
    timeSpent = [0 for _ in range(H)]   # checks if hallHours are exceeded
    for (course,time,hall) in schedule:
        timeSpent[hall-1] += courseDuration[course-1]
    for i in range(H):
        if timeSpent[i] > hallHours[i]:
            exceededTime = timeSpent[i] - hallHours[i]
            fitnessValue += 10*exceededTime
            if show:
                print("hallHours: ", hallHours, "timeSpent: ", timeSpent, "hall: ", i, "New Fitness: ", fitnessValue)
    return fitnessValue

# tournament slecetion of 5 with least fitness value and return the best
def selection(population):
    population = sorted(population, key=lambda x: fitness(x))
    return population[:5]

# perform single-point crossover on 2 schedules and return 2 new schedules
def performSinglePointCrossover(schedule1, schedule2,show=False):
    if show:
        print("Performing Single Point Crossover on: ")
        print(schedule1)
        print(schedule2)
    pointOfCrossover = random.randint(1,N-1)
    generatedSchedule1 = schedule1[:pointOfCrossover] + schedule2[pointOfCrossover:]
    generatedSchedule2 = schedule2[:pointOfCrossover] + schedule1[pointOfCrossover:]
    if show:
        print("Generated Schedules after crossover: ")
        print(generatedSchedule1)
        print(generatedSchedule2)
    return generatedSchedule1, generatedSchedule2

# perform single point mutation on a schedule and return the new schedule
def performMutation(schedule,show=False):
    if show:
        print("Performing Mutation on: ")
        print(schedule)
    tempSchedule = deepcopy(schedule)
    pointOfMutation = random.randint(0,N-1)
    # course = random.randint(1,N)
    time = random.randint(1,K)
    hall = random.randint(1,H)
    tempSchedule[pointOfMutation] = (pointOfMutation+1,time,hall)
    if show:
        print("Generated Schedule after mutation: ")
        print(tempSchedule)
    return tempSchedule

#  match schedules
def match(schedule1, schedule2):
    for (course1, time1, hall1) in schedule1:
        for (course2, time2, hall2) in schedule2:
            if course1 == course2 and time1 == time2 and hall1 == hall2:
                return True
    return False

# stopping criteria
def stopingCriteria(population,generation,perfectSolutions):
    for schedule in population:
        if fitness(schedule) == 0: # and generation < generationLimit:
            if len(perfectSolutions) == 0:
                perfectSolutions.append(schedule)
            for perfectSolution in perfectSolutions:
                if match(schedule, perfectSolution) == True:
                    # return False 
                    return True   
                perfectSolutions.append(schedule)
            # return False
            return True
        # elif fitness(schedule) == 0 and generation >= generationLimit:
        #     if len(perfectSolutions) == 0:
        #         perfectSolutions.append(schedule)
        #     for perfectSolution in perfectSolutions:
        #         if match(schedule, perfectSolution)==True:
        #             return True
        #         perfectSolutions.append(schedule)
        #     return True
    return False

# genetic algorithm
def geneticAlgorithm(initialPopulationSize,perfectSolutions,generationLimit):
    population = generatePopulation(initialPopulationSize)
    generation = 0
    while not stopingCriteria(population,generation,perfectSolutions) and generation < generationLimit:
        newPopulation = []
        # for i in range(initialPopulationSize):
        tournamentSelectedSchedule = selection(population)
        for schedule in tournamentSelectedSchedule:
            crossoverProbability = random.randint(1,10)
            if crossoverProbability <= 8 and len(tournamentSelectedSchedule)>1:
                print("Performing Crossover")
                parent2 = []
                parent2 = random.choice(tournamentSelectedSchedule)
                randomCount = 0
                while parent2==schedule and randomCount<10:
                    parent2 = random.choice(tournamentSelectedSchedule) 
                    randomCount += 1
                newSchedule1, newSchedule2 = performSinglePointCrossover(schedule, parent2,show=True)
                if schedule not in newPopulation:
                    newPopulation.append(schedule)
                if parent2 not in newPopulation:
                    newPopulation.append(parent2)
                newPopulation.append(newSchedule1)
                newPopulation.append(newSchedule2)
            if crossoverProbability>9:
                print("Performing Mutation")
                mutatedSchedule = performMutation(schedule,show=True)
                newPopulation.append(mutatedSchedule)
                if schedule not in newPopulation:
                    newPopulation.append(schedule)
        # newPopulation = list(newPopulation)
        tempPopulation = []
        tempPopulation = selection(newPopulation)
        population = newPopulation
        if population == []:
            print("Population died out regenerating population using tournament: ")
            population = tournamentSelectedSchedule
            tempPopulation = tournamentSelectedSchedule
        global minimumSolution
        minimumSolution = tempPopulation[0]
        # print("Population after generation: ", generation)
        # printPopulation(population)
        generation += 1
        print("Generation: ", generation)
    return perfectSolutions

In [5]:
# population = generatePopulation(100)
print("Note indexing is set to start from 1")
print("Course Duration: ", courseDuration)
print("Hall Hours: ", hallHours)
print("Conflicts: ", conflicts)
print("Conflicts with Student Count: ", conflicts_With_StudentCount)
print("\n-----------------------------------------------\n")
print("Enter population size: ")
populationSize = int(input())
# print("Enter generation limit: ")
# generationLimit = int(input())
perfectSolutions = [] #list of perfect solutions with fitness 0
generationLimit = 100
perfectSolutions =  geneticAlgorithm(populationSize,perfectSolutions,generationLimit)
print("\n-----------------------------------------------\n") 
print(f"\n{len(perfectSolutions)} Perfect Solutions (fitness 0): ")
printPopulation(perfectSolutions)
if len(perfectSolutions) == 0:
    print("No perfect solution found")      
    print("Closest solution: ", minimumSolution)
    print("Minimum Fitness: ", fitness(minimumSolution))  

Note indexing is set to start from 1
Course Duration:  [4, 2, 3, 4, 4]
Hall Hours:  [10, 10]
Conflicts:  [(1, 2), (1, 4), (2, 5), (3, 4), (4, 5)]
Conflicts with Student Count:  [(1, 2, 10), (1, 4, 5), (2, 5, 7), (3, 4, 12), (4, 5, 8)]

-----------------------------------------------

Enter population size: 

-----------------------------------------------


1 Perfect Solutions (fitness 0): 
[(1, 3, 2), (2, 2, 2), (3, 3, 1), (4, 1, 2), (5, 1, 1)]
